In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tabulate import tabulate

def extrair_dados_miracle_items(soup_categoria):
    try:
        data = []

        # Encontrar o elemento h3 com a classe 'titlearea'
        title_area = soup_categoria.find('h3', class_='titlearea')

        # Se o elemento for encontrado, começar a busca a partir dele
        if title_area:
            # Encontrar a tabela dentro da seção
            table = title_area.find_next('div', class_='table-responsive')

            # Verificar se a tabela foi encontrada
            if table:
                for row in table.find('table').find_all('tr')[1:]:
                    columns = row.find_all('td')
                    # Adiciona verificação para garantir que há colunas suficientes
                    if len(columns) >= 5:
                        icon = columns[0].find('img')['src']
                        item_name = columns[1].get_text(strip=True)
                        item_type = columns[2].get_text(strip=True)
                        effect = columns[3].get_text(strip=True)
                        location = columns[4].get_text(strip=True)

                        data.append({
                            'Icon': icon,
                            'Item Name': item_name,
                            'Item Type': item_type,
                            'Effect': effect,
                            'Location': location,
                        })

                micracle_items_df = pd.DataFrame(data)

                return micracle_items_df
            else:
                raise Exception("Tabela não encontrada na seção Miracle Enhancement Items.")
        else:
            raise Exception("Elemento <h3> com classe 'titlearea' não encontrado.")

    except Exception as e:
        print(f"Erro ao extrair dados da categoria micracle_items: {e}")
        return None

# URL da categoria "micracle_items"
url_micracle_items = "https://darksouls3.wiki.fextralife.com/Miracles"

# Faz a requisição HTTP para a página da categoria
response_micracle_items = requests.get(url_micracle_items)

if response_micracle_items.status_code == 200:
    html_micracle_items = response_micracle_items.text
    soup_micracle_items = BeautifulSoup(html_micracle_items, 'html.parser')  # Use o parser padrão aqui

    # Extrai os dados da categoria "micracle_items"
    micracle_items_df = extrair_dados_miracle_items(soup_micracle_items)

    if micracle_items_df is not None and not micracle_items_df.empty:
        print("Dados da categoria micracle_items:")
        # Exibe os dados como tabela
        print(tabulate(micracle_items_df, headers='keys', tablefmt='pretty', showindex=False))
    else:
        print("Falha ao extrair dados específicos da categoria micracle_items ou o DataFrame está vazio.")
else:
    print(f"Falha ao obter a página da categoria. Código de status: {response_micracle_items.status_code}")
